In [1]:
!uv pip install pysam

Using Python 3.12.7 environment at: /Users/madhavajay/dev/biosynth/main/.venv
Resolved 1 package in 337ms                                          
Prepared 1 package in 2.15s                                              
Installed 1 package in 5ms                                  
 + pysam==0.23.3


In [2]:
import pysam


In [3]:
!ls data

GCF_000001405.40.gz     GCF_000001405.40.gz.md5 GCF_000001405.40.gz.tbi


In [4]:
vf = pysam.VariantFile("data/GCF_000001405.40.gz")

In [5]:
vf

In [6]:
!bcftools query -i 'ID=="rs7537756"' -f '%CHROM\t%POS\t%ID\t%REF\t%ALT\n' data/GCF_000001405.40.gz

NC_000001.11	918870	rs7537756	A	G


In [7]:
# rs7537756	1	918870	AA	0.8847	0	-0.1875

In [9]:
!pip install pandas

In [10]:
import pandas as pd

In [11]:
contig = "NC_000001.11"   # chr1 on GRCh38/dbSNP
pos    = 918_870          # 1-based POS from your line
target = "rs7537756"

hits = []
for rec in vf.fetch(contig, pos-1, pos):  # half-open: [pos-1, pos)
    if rec.id == target:
        hits.append({
            "id": rec.id,
            "contig": rec.contig,
            "pos": rec.pos,
            "ref": rec.ref,
            "alt": ",".join(rec.alts or []),
            "qual": rec.qual,
            "filters": ";".join(rec.filter.keys()) if rec.filter is not None else "PASS",
        })

if hits:
    display(pd.DataFrame(hits))
else:
    print("Not found exactly at position; trying a small ±50bp window…")
    for rec in vf.fetch(contig, pos-50, pos+50):
        if rec.id == target:
            print(rec.id, rec.contig, rec.pos, rec.ref, rec.alts)

,id,contig,pos,ref,alt,qual,filters
0,rs7537756,NC_000001.11,918870,A,G,None,
